In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Funktion zum Plotten**<br>
Die Funktion compare_acc_gyr erhält Daten aus den Accelerometer- und Gyroscope-DataFrames und plottet diese.<br>
Als erstes wird ein neuer subplot erstellt.<br>
Danach wird für jede Achse ein Scatter-Plot angelegt.<br>
Abschließend wird das Layout des Plots aktualisiert und ausgegeben.

In [ ]:
def compare_acc_gyr(acc_df,gyr_df):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('Accelerometer', 'Gyroscope'), vertical_spacing=0.1, row_heights=[3, 3, 3])
    # Subplot 1: Accelerometer-Daten
    fig.add_trace(go.Scatter(x=acc_df['Time (s)'], y=acc_df['Linear Acceleration x (m/s^2)'], name='Accel X'))
    fig.add_trace(go.Scatter(x=acc_df['Time (s)'], y=acc_df['Linear Acceleration y (m/s^2)'], name='Accel Y'))
    fig.add_trace(go.Scatter(x=acc_df['Time (s)'], y=acc_df['Linear Acceleration z (m/s^2)'], name='Accel Z'))

    # Subplot 2: Gyroscope-Daten
    fig.add_trace(go.Scatter(x=gyr_df['Time (s)'], y=gyr_df['Gyroscope x (rad/s)'], marker=dict(size=4), name='Gyr X'), row=2, col=1)
    fig.add_trace(go.Scatter(x=gyr_df['Time (s)'], y=gyr_df['Gyroscope y (rad/s)'], marker=dict(size=4), name='Gyr Y'), row=2, col=1)
    fig.add_trace(go.Scatter(x=gyr_df['Time (s)'], y=gyr_df['Gyroscope z (rad/s)'], marker=dict(size=4), name='Gyr Z'), row=2, col=1)

    # Aktualisiere Layout und Labels
    fig.update_layout(
                    xaxis_title='Zeit (s)',
                    yaxis_title='Wert',
                    legend_title='Achse',
                    height=800, width=1000)
    fig.show()

**Fast Fourier Transformation**<br>
Die Funktion plot_frequencies nutzt die Fast Fourier Transformation um die Frequenzen von Daten aus dem Dataframe zu analysieren und darzustellen.<br>
Es wird ein neuer Plot erstellt.<br>
Die Spalte Time (s) wird aus dem Dataframe entfernt da sie für die Frequenzanalyse nicht benötigt wird.<br>
Danach wird die Anzahl der verbleibenden Spalten im gefilterten Dataframe ermittelt.<br>
Als nächstes wird eine Schleife durchlaufen um die Frequenzanalyse für jede Spalte im Dataframe durchzuführen.<br>
Die Amplituden werden normalisiert in dem sie durch den Maximalen Amplitudenwert geteilt werden.<br>
Es werden nur die positiven Frequenzen betrachtet und die entsprechenden Werte werden extrahiert.<br>
Als letztes werden die einzelnen Subplots erstellt wobei für jede Spalte im Dataframe ein eigener Subplot erstellt wird.<br>
Es wird in jedem Subplot die Frequenz gegen die normalisierte Amplitude aufgetragen.<br>
Abschließend wird mit tight_layout gewährleistet das die Subplots ordentlich angeordnet sind um Überlappungen zu vermeiden und die Plots werden ausgegeben.<br>

In [ ]:
def plot_frequencies(dataframe):
    plt.figure(figsize=(15, 5))

    filtered_df = dataframe.drop(columns=['Time (s)'])
    num_columns = len(filtered_df.columns)

    for i, column in enumerate(filtered_df.columns, 1):
        dataframe_col = filtered_df[column]
        dft = np.fft.fft(dataframe_col)
        frequencies = np.fft.fftfreq(len(dataframe_col))
        amplitudes = np.abs(dft)

        # Normalisierung der Amplituden
        max_amplitude = np.max(amplitudes)
        normalized_amplitudes = amplitudes / max_amplitude

        # Filtern für positive Frequenzen
        positive_indices = np.where(frequencies > 0)
        positive_frequencies = frequencies[positive_indices]
        positive_normalized_amplitudes = normalized_amplitudes[positive_indices]

        # Erstellen der Subplots
        plt.subplot(1, num_columns, i)
        plt.plot(positive_frequencies, positive_normalized_amplitudes)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Normalized Amplitude')
        plt.title(f"Frequency {column}")

    plt.tight_layout()
    plt.show()

**Funktion zum Plotten der gefilterten Daten**<br>
Die Funktion plot_df nimmt ein Dataframe und gibt am Ende ein Plot aus.<br>
Es werden zwei Subplots erstellt um jeweils die Lineare Beschleunigung auf der x und y -Achse des Dataframes zu visiualisieren.<br>
Es wird ein weiterer Subplot erstellt um die Daten des Gyroscopes auf der x-Achse zu visiualisieren.

In [ ]:
def plot_df(df):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('Accelerometer', 'Gyroscope'), vertical_spacing=0.1, row_heights=[3, 3, 3])
    # Subplot 1: Accelerometer-Daten
    fig.add_trace(go.Scatter(x=df['Time (s)'], y=df['Linear Acceleration y (m/s^2)'], name='Accel Y'))
    fig.add_trace(go.Scatter(x=df['Time (s)'], y=df['Linear Acceleration z (m/s^2)'], name='Accel Z'))

    # Subplot 2: Gyroscope-Daten
    fig.add_trace(go.Scatter(x=df['Time (s)'], y=df['Gyroscope x (rad/s)'], marker=dict(size=4), name='Gyr X'), row=2, col=1)


    # Aktualisiere Layout und Labels
    fig.update_layout(
                    xaxis_title='Zeit (s)',
                    yaxis_title='Wert',
                    legend_title='Achse',
                    height=800, width=1000)
    fig.show()

In [ ]:
def plot_windows_with_data(df, windows):
    # Farben für die Fenster in Accel Y definieren
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'cyan', 'magenta', 'yellow']

    # Erstellen des Plots mit Plotly
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('Accel Y', 'Gyro X', 'Accel Z'))

    # Hinzufügen der Accelerometer Y-Daten (ohne Peaks)
    fig.add_trace(go.Scatter(x=df['Time (s)'], y=df['Linear Acceleration y (m/s^2)'], name='Accel Y', line=dict(color='grey')), row=1, col=1)

    # Hervorheben der Fenster in Accel Y mit verschiedenen Farben
    for i, window in enumerate(windows):
        window_color = colors[i % len(colors)]
        fig.add_trace(go.Scatter(x=window['Time (s)'], y=window['Linear Acceleration y (m/s^2)'], mode='lines', line=dict(color=window_color, width=2)), row=1, col=1)

    # Hinzufügen der Gyroscope X-Daten
    fig.add_trace(go.Scatter(x=df['Time (s)'], y=df['Gyroscope x (rad/s)'], name='Gyro X', line=dict(color='blue')), row=2, col=1)

    # Hinzufügen der Linear Acceleration Z-Daten
    fig.add_trace(go.Scatter(x=df['Time (s)'], y=df['Linear Acceleration z (m/s^2)'], name='Accel Z', line=dict(color='green')), row=3, col=1)

    # Update layout and labels
    fig.update_layout(title='Accelerometer Daten mit farblichen Fenstern in Accel Y', xaxis_title='Zeit (s)', height=900, width=1000)
    fig.show()
